In [2]:
# @title Data retrieval
import os, requests

fname = 'fingerflex.npz'
url = "https://osf.io/5m47z/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [3]:
from matplotlib import rcParams
from matplotlib import pyplot as plt

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] = 15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

# @title Data loading
import numpy as np

alldat = np.load(fname, allow_pickle=True)['dat']
# un arreglo de largo 1 tienen un arreglo de largo 3 con diccionarios 
# así nos quedamos con el arreglo de largo 3 con diccionarios
alldat = alldat[0]
dat = alldat[0]



In [17]:
%%time
from scipy import signal

# Preprocesamiento de señales para todos los sujetos
for i, dat in enumerate(alldat):
    V = dat['V'].astype('float32')
    
    # Filtro de paso alto para eliminar frecuencias bajas (por debajo de 50 Hz)
    b, a = signal.butter(3, 50, btype='high', fs=1000)
    V = signal.filtfilt(b, a, V, 0)
    
    # Convertir a potencia de señal
    V = np.abs(V)**2
    
    # Filtro de paso bajo para suavizar la señal (por debajo de 3 Hz)
    b, a = signal.butter(3, 3, btype='low', fs=1000)
    V = signal.filtfilt(b, a, V, 0)
    
    # Guardar los datos preprocesados en la estructura original
    dat['V_preprocessed'] = V

CPU times: total: 6.78 s
Wall time: 15.4 s


In [31]:
print(dat['stim_id'])
print(dat['stimtext'])

[4 5 4 2 4 1 3 3 3 3 2 5 3 4 4 3 4 1 2 4 3 4 2 1 4 5 4 4 1 3 4 1 2 5 3 5 3
 2 1 5 4 5 3 2 1 3 1 3 3 3 4 3 1 1 5 4 3 4 1 5 1 1 2 2 3 1 2 1 4 4 1 2 2 1
 4 2 5 3 2 4 5 3 1 4 4 5 5 3 5 4 4 3 4 5 2 2 5 2 3 5 2 5 1 4 5 3 5 3 4 3]
['thumb', 'fore', 'middle', 'fourth', 'pinky']


In [21]:
%%time
import pandas as pd

# Inicializar una lista para almacenar las observaciones
data = []

# Iterar sobre todos los sujetos
for subject_id, dat in enumerate(alldat):
    V = dat['V']
    V_preprocessed = dat['V_preprocessed']
    dg = dat['dg']
    t_on = dat['t_on']
    t_off = dat['t_off']
    stim_id = dat['stim_id']
    
    # Iterar sobre todos los cues
    for i in range(len(t_on)):
        # Extraer el segmento de la señal durante el cue
        start = t_on[i]
        end = t_off[i]
        
        # Asegurarse de que los índices son enteros
        start = int(start)
        end = int(end)
        
        # Extraer la señal de voltaje y la señal de flexión de dedos para el segmento
        V_segment = V[start:end, :]
        dg_segment = dg[start:end, :]
        V_preprocessed_segment = V_preprocessed[start:end, :]
        
        # Crear una observación con toda la información relevante
        observation = {
            'subject': subject_id,
            'stim_id': stim_id[i],
            'V': V_segment,
            'dg': dg_segment,
            'V_preprocessed': V_preprocessed_segment
        }
        
        # Añadir la observación a la lista
        data.append(observation)

# Convertir la lista de observaciones en un DataFrame de pandas
df = pd.DataFrame(data)
df

CPU times: total: 0 ns
Wall time: 4.75 ms


,subject,stim_id,V,dg,V_preprocessed
0,0,5,"[[-0.01025, -0.0871, 0.04306, 0.1816, -0.707, ...","[[1.0, 2.0, 2.0, 1.0, 0.0], [1.0, 2.0, 2.0, 1....","[[0.006731084950751146, 0.002257720094176623, ..."
1,0,5,"[[-0.177, -0.2876, 0.07544, 0.1895, 0.3037, 0....","[[2.0, 36.0, 122.0, 9.0, 0.0], [2.0, 36.0, 122...","[[0.004354213243907625, 0.0018563630848983236,..."
2,0,5,"[[-1.365, 0.9497, 0.0689, 0.0918, -1.464, -0.9...","[[16.0, 60.0, 77.0, 3.0, 27.0], [16.0, 60.0, 7...","[[0.008742187758238663, 0.003457486939732531, ..."
3,0,1,"[[-0.3464, 0.7417, 0.9146, 1.17, 0.1941, -0.37...","[[44.0, 39.0, 53.0, 0.0, 52.0], [44.0, 39.0, 5...","[[0.00421649839000787, 0.0040001668756106615, ..."
4,0,5,"[[0.3442, -0.313, -0.5215, -1.045, -0.7725, 0....","[[47.0, 17.0, 133.0, 31.0, 9.0], [47.0, 17.0, ...","[[0.0036529056903043273, 0.003316106786961743,..."
...,...,...,...,...,...
405,2,3,"[[-0.03857, 0.502, 0.2104, 0.7744, -0.6226, -2...","[[0.0, 1.0, 0.0, 2.0, 0.0], [0.0, 1.0, 0.0, 2....","[[0.00764382774334796, 0.02064491755229973, 0...."
406,2,5,"[[0.2458, -0.007088, -1.508, -1.801, 1.051, -0...","[[-20.0, 260.0, 33.0, 479.0, 353.0], [0.0, 260...","[[0.017938702121421377, 0.0383926242430772, 0...."
407,2,3,"[[-1.474, 0.2651, 0.62, 1.344, -0.617, 0.567, ...","[[62.0, 387.0, 76.0, 611.0, 477.0], [62.0, 387...","[[0.030677268141299438, 0.03174991293925156, 0..."
408,2,4,"[[-1.951, -1.415, -1.924, -2.309, -2.096, -0.9...","[[40.0, 15.0, 59.0, 24.0, -4.0], [40.0, 15.0, ...","[[0.10105765948144241, 0.13231091903095898, 0...."


In [23]:
sample_obs = df.iloc[0]
print(f"Sujeto: {sample_obs['subject']}, Stim_id: {sample_obs['stim_id']}")
print(f"Forma de V_segment: {sample_obs['V'].shape}")
print(f"Forma de V_preprocessed_segment: {sample_obs['V_preprocessed'].shape}")
print(f"Forma de dg_segment: {sample_obs['dg'].shape}")

Sujeto: 0, Stim_id: 5
Forma de V_segment: (2000, 84)
Forma de V_preprocessed_segment: (2000, 84)
Forma de dg_segment: (2000, 5)


In [41]:
%%time
from dtaidistance import dtw
from scipy.stats import zscore
import numpy as np
from tqdm import tqdm

def calculate_channel_distances(V, dg):
    distances = []
    for channel in range(V.shape[1]):
        signal1 = zscore(V[:, channel])
        signal2 = zscore(dg[:, channel % dg.shape[1]])  # Normalizamos también dg por canal correspondiente
        distance = dtw.distance(signal1, signal2)
        distances.append(distance)
    return np.array(distances)

# Añadir columnas al DataFrame original con barra de progreso
tqdm.pandas()  # Añadir soporte para barras de progreso a pandas

df['dist_v_dg'] = df.progress_apply(lambda row: calculate_channel_distances(row['V'], row['dg']), axis=1)
df['dist_vpre_dg'] = df.progress_apply(lambda row: calculate_channel_distances(row['V_preprocessed'], row['dg']), axis=1)

# Verificar que las nuevas columnas se han añadido correctamente
print(df[['dist_v_dg', 'dist_vpre_dg']].head())
df

  0%|          | 0/410 [00:00<?, ?it/s]

  1%|          | 4/410 [1:13:42<135:48:31, 1204.22s/it]

In [39]:
# set seed 
np.random.seed(420)

# Seleccionar aleatoriamente una observación por dedo para cada sujeto
selected_observations = []

for subject_id in range(df['subject'].nunique()):
    for stim_id in range(1,6):  # Hay 5 dedos
        # Filtrar las observaciones del sujeto actual y del dedo actual
        filtered_df = df[(df['subject'] == subject_id) & (df['stim_id'] == stim_id)]
        if not filtered_df.empty:
            # Seleccionar una observación aleatoria
            selected_observation = filtered_df.sample(n=1).iloc[0]
            selected_observations.append(selected_observation)

# Convertir la lista de observaciones seleccionadas en un DataFrame
selected_df = pd.DataFrame(selected_observations)
selected_df

,subject,stim_id,V,dg,V_preprocessed
50,0,1,"[[-1.733, -1.211, -1.241, -0.776, -0.3848, 1.1...","[[1.0, 34.0, 19.0, 25.0, 13.0], [1.0, 34.0, 19...","[[0.003225941103414855, 0.003627645240877666, ..."
31,0,2,"[[0.6006, 0.9517, 0.0785, -0.2502, -0.6943, 0....","[[15.0, 52.0, 26.0, 40.0, 28.0], [15.0, 52.0, ...","[[0.0033715521884203972, 0.005533692269513891,..."
86,0,3,"[[-0.2172, -0.2524, -0.453, -1.495, 0.0796, -0...","[[75.0, 520.0, 9.0, 52.0, 45.0], [75.0, 520.0,...","[[0.00542449796380343, 0.0017684900873680411, ..."
114,0,4,"[[-0.5728, 1.251, 1.591, -0.1235, -1.993, -1.9...","[[2.0, 1.0, 17.0, 0.0, 8.0], [2.0, 1.0, 17.0, ...","[[0.005987668304397922, 0.0020905908815570737,..."
131,0,5,"[[-1.235, 0.4595, 0.987, -0.02913, -0.632, -0....","[[2.0, 30.0, 15.0, 14.0, 5.0], [2.0, 30.0, 15....","[[0.008034979164468856, 0.002820028055154099, ..."
152,1,1,"[[0.532, 1.172, 0.4854, -0.9326, -0.6636, -0.5...","[[17.0, 130.0, 124.0, 54.0, 71.0], [17.0, 130....","[[0.00045805029783141653, 0.000467019694395020..."
170,1,2,"[[-0.01804, 0.3584, 0.4749, 0.597, 0.522, 0.70...","[[9.0, 16.0, 62.0, 2.0, 51.0], [9.0, 16.0, 62....","[[0.0009081612479327353, 0.0010670425556953951..."
295,1,3,"[[-1.171, -0.1976, 1.053, 0.6743, 0.2097, 0.06...","[[7.0, 36.0, 223.0, 68.0, 48.0], [7.0, 36.0, 2...","[[0.0009482079790527665, 0.0008162263165839536..."
166,1,4,"[[-0.7036, -0.4126, -0.3289, 0.10516, 1.261, 0...","[[20.0, 40.0, 192.0, 35.0, 102.0], [20.0, 40.0...","[[0.0004594822529055494, 0.0005854965838166262..."
180,1,5,"[[-0.7646, -2.574, -1.715, -1.204, -1.574, -0....","[[14.0, 1.0, 139.0, 20.0, 21.0], [14.0, 1.0, 1...","[[0.00035284964463178943, 0.000250878084222813..."
